In [11]:
from transformers import pipeline
import torch 

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = pipeline('text-generation', model='gpt2-xl')
# mrm8488/GPT-2-finetuned-covid-bio-medrxiv
# stanford-crfm/pubmed_gpt
# chrisliu298/arxiv_ai_gpt2

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.99G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [12]:
def query(text):
    outputs = generator(
        text,
        return_full_text=False,
        max_token_length=256,
        max_length=128,
        pad_token_id=generator.tokenizer.eos_token_id,
        do_sample=True
    )
    return outputs

In [14]:
import numpy as np

def construct_passage(first_sentence, max_num_sentences=10):
    """
    Construct a passage from the first sentence.
    """
    passage_all = first_sentence
    range_var = np.random.randint(4, max_num_sentences)
    for _ in range(range_var):
        passage_that_fits = passage_all[-256:]
        
        data = query(passage_that_fits)
        passage_all += data[0]["generated_text"].replace('\n','')
    return passage_all.replace(first_sentence, '')

print(construct_passage("Directional distance functions provide very flexible tools for investigating the performance of Decision Making Units (DMUs)."))

 For more information on this topic, see DMU Basics.There are five elements within directional distance and three functions. The Directional Distance FunctionAn Example of Directional Distance FunctionA two dimensional vector is represented by a vector of length two, where two contains all of the individual points. The following diagram illustrates how a two dimensional vector is represented by this five-dimensional Vector of Length Two.The first thing to notice is that our two dimensional vector is not an ideal representation of our 2D vector. What we have before us is a vector of length two, and this is actually an approximation of the vector of length four. You don't get the full four dimensional result that you would if you added the dimension of the first component to the second component.In terms of a two dimensional vector, we have some properties like the direction, the magnitude, the direction of acceleration, and the magnitude, the angle of the acceleration. We can say that w

In [15]:
import pandas as pd
df = pd.read_csv(('./real_passages_100k.csv'))
samples = df.sample(n=1000, random_state=16).fillna("")

In [16]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from tqdm.auto import tqdm
from csv import writer

passages = []
j = 0
for i, sample in tqdm(samples.iterrows(), total=len(samples)):
    new_passage = construct_passage(sample['passages'])
    passages.append(
        new_passage
    )

  0%|          | 0/1000 [00:00<?, ?it/s]

Input length of input_ids is 128, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 128, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 128, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 138, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 138, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 137, but ``max_length`` is set to 128. This can lead to unexpected behavior. You 

Input length of input_ids is 145, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 145, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 128, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 128, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 129, but ``max_length`` is set to 128. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 128, but ``max_length`` is set to 128. This can lead to unexpected behavior. You 

In [ ]:

from difflib import SequenceMatcher

SIMILARITY_THRESHOLD = 0.8

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
clean = []
for para, original in zip(passages, list(samples['passages'].fillna(''))):
    if para.strip() == '':
        continue
    if similar(para, original) <= SIMILARITY_THRESHOLD:
        clean.append(para)

In [ ]:
len(clean)

In [ ]:
df = pd.DataFrame({
    "passages": clean
})
df.to_csv('./GPT-xl_generate.csv', index=False)